In [7]:
import pandas as pd

In [8]:
data = pd.read_csv('aplicaciones_limpias2.csv')
data.info()
data = data.rename(columns={'Aplicación': 'Aplicacion'})
data['nivel'] = data['nivel'].map({1: 'Junior', 0: 'No Junior'})

<class 'pandas.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Aplicación     224 non-null    int64
 1   Empresa        224 non-null    str  
 2   Posicion       224 non-null    str  
 3   Fecha          224 non-null    str  
 4   url            206 non-null    str  
 5   Junta ARC      224 non-null    int64
 6   Canal_limpio   224 non-null    str  
 7   tipo_posicion  224 non-null    str  
 8   nivel          224 non-null    int64
dtypes: int64(3), str(6)
memory usage: 15.9 KB


In [9]:
#Busco hacer IDs únicos para cada empresa y cada aplicación. 
dim_canal2 = data[['Canal_limpio']].drop_duplicates().reset_index(drop=True)
dim_canal2['Canal_ID'] = range(1, len(dim_canal2) + 1)

dim_fecha2 = data[['Fecha']].drop_duplicates().reset_index(drop=True)
dim_fecha2['Fecha_ID'] = range(1, len(dim_fecha2) + 1)

dim_empresa2 = data[['Empresa']].drop_duplicates().reset_index(drop=True)
dim_empresa2['Empresa_ID'] = range(1, len(dim_empresa2) + 1)

dim_aplicacion2 = data[['Aplicacion']].drop_duplicates().reset_index(drop=True)
dim_aplicacion2['Aplicacion_ID'] = range(1, len(dim_aplicacion2) + 1)

dim_junta2 = data[['Junta ARC']].drop_duplicates().reset_index(drop=True)
dim_junta2['Junta_ID'] = range(1, len(dim_junta2) + 1)
#dim posiciones 
dim_posicion2 = data[['tipo_posicion', 'nivel']].drop_duplicates().reset_index(drop=True)
dim_posicion2['Posicion_ID'] = range(1, len(dim_posicion2) + 1)
dim_posicion2['nivel'] = dim_posicion2['nivel'].map({1: 'Junior', 0: 'No Junior'})

#Renombrar columna con tilde 
dim_aplicacion2 = dim_aplicacion2.rename(columns={'Aplicación': 'Aplicacion'})

display(dim_canal2.head())
display(dim_fecha2.head())
display(dim_empresa2.head())
display(dim_aplicacion2.head())
display(dim_posicion2.head())

,Canal_limpio,Canal_ID
0,LinkedIn,1
1,Otros,2
2,CT,3
3,Glassdoor,4
4,Indeed,5


,Fecha,Fecha_ID
0,2025-10-22,1
1,2025-10-29,2
2,2025-11-05,3
3,2025-11-12,4
4,2025-11-19,5


,Empresa,Empresa_ID
0,Idis,1
1,Amaris,2
2,Smilepill,3
3,Adam Milo,4
4,Incode,5


,Aplicacion,Aplicacion_ID
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


,tipo_posicion,nivel,Posicion_ID
0,Otros,NaN,1
1,Data Scientist,NaN,2
2,Data Scientist,NaN,3
3,Data Analyst,NaN,4
4,Data Analyst,NaN,5


In [10]:
fact_aplicaciones2 = (
    data
    .merge(dim_canal2, how='left', on='Canal_limpio').merge(dim_fecha2, how='left', left_on='Fecha', right_on='Fecha') 
    .merge(dim_empresa2, how='left', on='Empresa') 
    .merge(dim_aplicacion2, how='left', on='Aplicacion')  
    .merge(dim_junta2, how='left', on='Junta ARC')
    .merge(dim_posicion2, how='left', on=['tipo_posicion', 'nivel']))

#Seleccionar columnas para tabla de hechos 
hechos_aplicaciones = fact_aplicaciones2[['Canal_ID', 'Fecha_ID', 'Empresa_ID', 'Aplicacion_ID', 'Junta_ID', 'Posicion_ID']]

In [11]:

#Retirar columnas de dimensiones 
fact_aplicaciones2 = fact_aplicaciones2.drop(columns=['Canal_limpio', 'Empresa', 'Junta ARC', 'tipo_posicion', 'nivel', 'Posicion'])

#Tabla de hechos. ID único para cada conexión. 
display(fact_aplicaciones2)

,Aplicacion,Fecha,url,Canal_ID,Fecha_ID,Empresa_ID,Aplicacion_ID,Junta_ID,Posicion_ID
0,1,2025-10-22,NaN,1,1,1,1,1,NaN
1,2,2025-10-22,NaN,1,1,2,2,1,NaN
2,3,2025-10-22,NaN,1,1,3,3,1,NaN
3,4,2025-10-22,NaN,1,1,4,4,1,NaN
4,5,2025-10-29,https://job-boards.greenhouse.io/incode/jobs/5...,2,2,5,5,2,NaN
...,...,...,...,...,...,...,...,...,...
219,227,2025-12-05,https://www.glassdoor.com.mx/partner/jobListin...,4,7,191,220,6,NaN
220,228,2025-12-05,https://torre.ai/post/GdjqEJXw-simera-data-eng...,7,7,192,221,6,NaN
221,229,2025-12-05,https://www.linkedin.com/jobs/view/4368650483/...,1,7,193,222,6,NaN
222,230,2025-12-05,https://www.linkedin.com/jobs/view/4350246433/...,1,7,194,223,6,NaN


In [12]:
#Pasar a CSVs exportables para power bi 
dim_canal2.to_csv('dim_canal2.csv', index=False)
dim_junta2.to_csv('dim_junta2.csv', index=False)
dim_fecha2.to_csv('dim_fecha2.csv', index=False) 
dim_empresa2.to_csv('dim_empresa2.csv', index=False)
dim_aplicacion2.to_csv('dim_aplicacion2.csv', index=False)
dim_posicion2.to_csv('dim_posicion2.csv', index=False)
fact_aplicaciones2.to_csv('fact_aplicaciones2.csv', index=False)